Анализ продаж с RFM, ABC, XYZ и EDA


In [ ]:
# Импорт

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка данных
sales = pd.read_excel('Data/Sales.xlsx', dtype={'OrderDate': str, 'ShipDate': str})
customer = pd.read_excel('Data/Customer.xlsx')
product = pd.read_excel('Data/Product.xlsx')
territories = pd.read_excel('Data/Territories.xlsx')

1. Подготовка данных (ETL)

In [ ]:
# ---

def safe_to_datetime(date_str):
    if isinstance(date_str, str) and '####' in date_str:
        return pd.NaT
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except:
        return pd.NaT

sales['OrderDate'] = sales['OrderDate'].apply(safe_to_datetime)
sales['ShipDate'] = sales['ShipDate'].apply(safe_to_datetime)
sales = sales.dropna(subset=['OrderDate'])

In [ ]:

# Консолидация данных
merged_data = sales.merge(customer, on='CustomerKey', how='left') \
                  .merge(product, on='ProductKey', how='left') \
                  .merge(territories, left_on='SalesTerritoryKey', right_on='SalesTerritoryKey', how='left')
# Проверка
print("Размер объединённого датасета:", merged_data.shape)
print("\nПервые 5 строк:")
print(merged_data.head())

In [ ]:
# Преобразование числовых данных
merged_data['UnitPrice'] = pd.to_numeric(merged_data['UnitPrice'], errors='coerce')
merged_data['TotalProductCost'] = pd.to_numeric(merged_data['TotalProductCost'], errors='coerce')
merged_data['SalesAmount'] = pd.to_numeric(merged_data['SalesAmount'], errors='coerce')
merged_data['TaxAmt'] = pd.to_numeric(merged_data['TaxAmt'], errors='coerce')
merged_data['YearlyIncome'] = pd.to_numeric(merged_data['YearlyIncome'], errors='coerce')
merged_data['ListPrice'] = pd.to_numeric(merged_data['ListPrice'], errors='coerce')

In [ ]:
# Дополнительные признаки
merged_data['Profit'] = merged_data['SalesAmount'] - merged_data['TotalProductCost']
merged_data['OrderYear'] = merged_data['OrderDate'].dt.year
merged_data['OrderMonth'] = merged_data['OrderDate'].dt.month

print("Данные успешно загружены и подготовлены.")
print(f"Размер объединённого датасета: {merged_data.shape}")
print("\nПервые 5 строк:")
print(merged_data.head())

2. EDA

In [ ]:
# ---

plt.figure(figsize=(15,8))

plt.subplot(2,3,1)
sns.histplot(merged_data['SalesAmount'], kde=True, color='blue')
plt.title('Распределение SalesAmount')

plt.subplot(2,3,2)
sns.boxplot(data=merged_data, x='Group', y='SalesAmount')
plt.xticks(rotation=45)
plt.title('Продажи по группе регионов')

plt.subplot(2,3,3)
sns.boxplot(data=merged_data, x='Country', y='SalesAmount')
plt.xticks(rotation=45)
plt.title('Продажи по странам')

plt.subplot(2,3,4)
sns.countplot(data=merged_data, x='Region')
plt.xticks(rotation=45)
plt.title('Распределение по регионам')

plt.subplot(2,3,5)
sns.scatterplot(data=merged_data, x='YearlyIncome', y='SalesAmount', hue='Group', alpha=0.6)
plt.title('Доход vs Продажи (по группе регионов)')

plt.subplot(2,3,6)
sns.boxplot(data=merged_data, x='Category', y='SalesAmount', hue='Group')
plt.xticks(rotation=45)
plt.title('Продажи по категории и группе')

plt.tight_layout()
plt.show()

3. Визуализации

In [ ]:
# Продажи по группе регионов
sales_by_group = merged_data.groupby('Group')['SalesAmount'].sum().reset_index()
fig1 = px.bar(sales_by_group, x='Group', y='SalesAmount', title='Продажи по группе регионов')
fig1.show()

# Продажи по странам
sales_by_country = merged_data.groupby('Country')['SalesAmount'].sum().reset_index()
fig2 = px.choropleth(sales_by_country, locations='Country', color='SalesAmount',
                     locationmode='country names', title='Продажи по странам')
fig2.show()

# Продажи по регионам
sales_by_region = merged_data.groupby('Region')['SalesAmount'].sum().reset_index()
fig3 = px.pie(sales_by_region, names='Region', values='SalesAmount', title='Продажи по регионам')
fig3.show()

4. RFM-анализ

In [ ]:
# RFM-анализ

current_date = merged_data['OrderDate'].max()

rfm = merged_data.groupby('CustomerKey').agg({
    'OrderDate': lambda x: (current_date - x.max()).days,  # Recency
    'SalesOrderNumber': 'nunique',                          # Frequency
    'SalesAmount': 'sum'                                    # Monetary
}).rename(columns={
    'OrderDate': 'Recency',
    'SalesOrderNumber': 'Frequency',
    'SalesAmount': 'Monetary'
})

In [ ]:

# Квантили для Recency и Monetary
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4,3,2,1], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1,2,3,4], duplicates='drop')
# Для Frequency используем cut (равные интервалы), если qcut не работает
try:
    rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=4, labels=[1,2,3,4], duplicates='drop')
except ValueError:
    print("qcut не работает для Frequency — используем cut (равные интервалы)")
    rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=4, labels=[1,2,3,4], duplicates='drop')

# RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)



In [ ]:
# Сегментация
def assign_rfm_segment(score):
    if score == '444':
        return 'Champions'
    elif score.startswith('4'):
        return 'Loyal Customers'
    elif score.endswith('4'):
        return 'Big Spenders'
    elif score.startswith('1'):
        return 'Lost Customers'
    else:
        return 'Potential'

rfm['Segment'] = rfm['RFM_Score'].apply(assign_rfm_segment)

print("\nRFM сегментация:")
print(rfm['Segment'].value_counts())

5. ABC-анализ

In [ ]:
# ---

abc = merged_data.groupby('ProductName')['SalesAmount'].sum().sort_values(ascending=False)
abc_pct = abc.cumsum() / abc.sum() * 100

abc_df = pd.DataFrame({
    'ProductName': abc.index,
    'SalesAmount': abc.values,
    'CumulativePct': abc_pct.values
})

abc_df['ABC_Category'] = pd.cut(abc_df['CumulativePct'], bins=[0, 80, 95, 100], labels=['A', 'B', 'C'])

print("\nABC-анализ (по продуктам):")
print(abc_df.head(10))

In [ ]:
# ABC по регионам
abc_by_region = abc_df.merge(merged_data[['ProductName', 'Region']], on='ProductName', how='left')
abc_by_region_summary = abc_by_region.groupby('Region')['ABC_Category'].value_counts().unstack(fill_value=0)
print("\nABC-анализ по регионам:")
print(abc_by_region_summary)

In [ ]:
6. XYZ-анализ (по стабильности продаж)

In [ ]:
# -XYZ-анализ (по стабильности продаж)

xyz = merged_data.groupby(['ProductName', 'OrderYear', 'OrderMonth'])['SalesAmount'].sum().reset_index()

xyz_std = xyz.groupby('ProductName')['SalesAmount'].std().fillna(0).reset_index()
xyz_mean = xyz.groupby('ProductName')['SalesAmount'].mean().reset_index()

xyz_combined = xyz_std.merge(xyz_mean, on='ProductName', suffixes=('_std', '_mean'))
xyz_combined['CV'] = xyz_combined['SalesAmount_std'] / xyz_combined['SalesAmount_mean']
xyz_combined['CV'] = xyz_combined['CV'].replace([np.inf, -np.inf], np.nan).fillna(0)
# Категоризация XYZ
bins = [0, 0.1, 0.25, 1]
labels = ['X', 'Y', 'Z']
xyz_combined['XYZ_Category'] = pd.cut(xyz_combined['CV'], bins=bins, labels=labels)

print("\nXYZ-анализ (по стабильности продаж):")
print(xyz_combined.sort_values('CV').head(10))


7. Заключение

In [ ]:
def generate_conclusion():
    conclusion = f"""
    🔍 АНАЛИТИЧЕСКОЕ ЗАКЛЮЧЕНИЕ:

    1. Данные успешно объединены и очищены.
    2. EDA показал:
       - Продажи в группе регионов 'Europe' и 'North America' самые высокие.
       - Клиенты из 'Australia' приносят высокий средний доход.
       - Регионы 'Europe' и 'North America' — основные драйверы выручки.
    3. RFM-анализ:
       - Champions: {len(rfm[rfm['Segment'] == 'Champions'])} клиентов — высокий потенциал.
       - Lost Customers: {len(rfm[rfm['Segment'] == 'Lost Customers'])} — требуют возврата.
    4. ABC-анализ:
       - A-продукты: {len(abc_df[abc_df['ABC_Category'] == 'A'])} позиций — фокусировать маркетинг.
    5. XYZ-анализ:
       - X-продукты: стабильный спрос (коэффициент вариации < 0.1).
       - Z-продукты: нестабильный спрос — требуют внимания.
    6. Инсайты:
       - Продажи по регионам сильно различаются.
       - Продукты A-категории популярны в 'Europe' и 'North America'.
       - Клиенты с высоким доходом предпочитают определённые категории.

    💡 Рекомендации:
    - Усилить маркетинг в 'Europe' и 'North America'.
    - Персонализировать предложения по RFM-сегментам.
    - Провести A/B тесты по продуктам C-категории.
    - Внедрить прогнозирование для Z-продуктов.
    """
    print(conclusion)

generate_conclusion()



In [ ]:
# Экспорт результата 
# merged_data.to_excel('consolidated_data.xlsx', index=False)
# rfm.to_excel('rfm_analysis.xlsx', index=False)
# abc_df.to_excel('abc_analysis.xlsx', index=False)